# Vision Transformer (ViT)
이번 실습에서는 Vision Transformer (ViT) 아키텍처를 직접 구현해보겠습니다. 

ViT는 자연어 처리(NLP)에서 주로 사용되던 <b>Transformer 아키텍처</b>를 이미지에 적용하여 이미지 분류 문제에서 뛰어난 성능을 발휘하는 모델입니다. 

이번 실습의 목표는 ViT를 구성하는 <b>핵심 모듈들을 단계별로 구현</b>해봄으로써, ViT 아키텍처의 구조와 동작 원리를 자세히 이해하는 것입니다.

ViT는 다음과 같은 모듈들로 구성되어 있습니다
- `ImagePatchifier` : 입력 이미지를 여러 개의 작은 패치(patch)로 분할합니다.
- `PatchEmbedding` : 분할된 이미지 패치를 벡터 형태의 임베딩으로 변환합니다. ViT에서는 각 이미지 패치가 문장의 단어(token)처럼 취급됩니다.
- `MultiHeadSelfAttention` : 입력 이미지의 여러 패치(patch)들 간의 Multi-head Self-attention을 계산합니다. 이를 통해 각 패치는 이미지 전체 맥락(context)을 반영한 더욱 풍부하고 정교한 표현(representation)을 학습합니다.
- `FeedForwardNetwork` : Self-Attention의 출력을 비선형 변환하여 더욱 복잡하고 풍부한 특징을 학습하게 합니다.
- `TransformerEncoderLayer` : `MultiHeadSelfAttention`과 `FeedForwardNetwork`를 결합한 하나의 Transformer 인코더 블록입니다.
- `ViT` : 위의 모든 모듈들을 종합하여 Vision Transformer 모델을 구현합니다.

Original paper : An Image is Worth 16x16 Words: Transformers for Image Recognition at Scale ([link](https://arxiv.org/pdf/2010.11929))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import torch
from torch import nn
from torchvision import transforms

from training_utilities import load_cifar10_dataloaders, train_one_epoch, evaluate_one_epoch

## Multi-Head Self-Attention

Transformer의 핵심 모듈로, 입력 토큰(token)들 간의 상호작용을 학습합니다. 이를 통해 각 토큰은 나머지 전체 토큰의 맥락(context)을 반영한 더욱 풍부하고 정교한 표현(representation)을 얻게 됩니다.


### Scaled Dot-Product Self-Attention
단일 head의 Self-Attention은 다음 수식을 통해 계산됩니다:

1. 입력 토큰 임베딩
   $$\mathbf{X} = (\mathbf{x}_1, \mathbf{x}_2, ..., \mathbf{x}_T) \in \mathbb{R}^{T \times d_{embed}}$$
2. 입력 임베딩을 선형변환하여 Query, Key, Value를 얻습니다.
   $$\mathbf{Q} = \mathbf{X} \mathbf{W}_Q \in \mathbb{R}^{T \times d_{k}},\quad
     \mathbf{K} = \mathbf{X} \mathbf{W}_K \in \mathbb{R}^{T \times d_{k}},\quad
     \mathbf{V} = \mathbf{X} \mathbf{W}_V \in \mathbb{R}^{T \times d_{v}}$$
   -  $\mathbf{W}_Q,\mathbf{W}_K \in \mathbb{R}^{d_{embed} \times d_{k}}, \mathbf{W}_V \in \mathbb{R}^{d_{embed} \times d_{v}}$ 는 학습 가능한 가중치 행렬입니다.
3. <b>Scaled Dot-product Self-attention</b>
   $$ \text{Attention}(\mathbf{Q}, \mathbf{K}, \mathbf{V}) = \text{softmax}(\frac{\mathbf{Q}\mathbf{K}^\top}{\sqrt{d_k}})\mathbf{V} \in \mathbb{R}^{T \times d_{v}}$$
   - 여기서 $d_k$는 key벡터의 차원의 크기입니다.

### Multi-head Self-Attention
Self-Attention을 여러 번 병렬로 수행하면, 서로 다른 관점에서의 다양한 관계를 학습할 수 있습니다. 이를 <b>Multi-head attention</b>이라고 합니다.

$$ \text{head}_i = \text{Attention}(\mathbf{X} \mathbf{W}_Q^{(i)}, \mathbf{X} \mathbf{W}_K^{(i)}, \mathbf{X} \mathbf{W}_V^{(i)}) \in \mathbb{R}^{T \times d_{v}}$$
$$ \text{MultiHeadSelfAttention}(\mathbf{X}) = \text{Concat}(\text{head}_1, ..., \text{head}_h)\mathbf{W}_O \in \mathbb{R}^{T \times d_{embed}}$$
 - $h$는 head의 수
 - $\mathbf{W}_Q^{(i)},\mathbf{W}_K^{(i)} \in \mathbb{R}^{d_{embed} \times d_{k}}, \mathbf{W}_V^{(i)} \in \mathbb{R}^{d_{embed} \times d_{v}}$ : head $i$의 선형 변환 행렬
 - $\mathbf{W}_O \in \mathbb{R}^{h \cdot d_v \times d_{embed}}$ is output linear projection matrix 

일반적으로 다음과 같은 관계를 만족하도록 모델을 구성합니다.
 - $d_k = d_v$
 - $d_{embed} = h \times d_k$

---
<mark>실습</mark> `MultiHeadSelfAttention`을 완성하세요.

1. Query, Key, Value projection: 입력을 선형변환하여 `Q`, `K`, `V`를 얻습니다.

2. multiple heads로 분할: `Q`, `K`, `V`를 `(batch_size, num_heads, seq_length, head_dim)`의 shape을 가지도록 변환합니다.
   - `embed_dim = head_dim * num_heads`의 관계를 이용하여, 선형변환된 벡터를 head 개수만큼 쪼개어 사용합니다.
   - `torch.Tensor.view` ([docs](https://pytorch.org/docs/stable/generated/torch.Tensor.view.html))를 이용하여 텐서를 reshape합니다.
   - `torch.permute` ([docs](https://pytorch.org/docs/stable/generated/torch.permute.html))를 이용하여 텐서의 차원 순서를 재배열합니다

1. Scaled Dot-Product Attention 계산
   - `Q`와 `K`의 scaled-dot dot-product를 계산하여 `attention_scores`를 계산합니다.
     - `torch.transpose` ([docs](https://pytorch.org/docs/stable/generated/torch.transpose.html))
     - `torch.matmul` ([docs](https://pytorch.org/docs/stable/generated/torch.matmul.html)) 함수는 각 텐서의 마지막 두 차원에 대한 행렬 곱을 수행하고, 앞의 차원은 batch차원으로 간주합니다.
     - `** 0.5` 연산을 이용하여 제곱근을 계산하세요
   - `torch.softmax` 함수를 이용하여 `attention_weights`를 계산합니다.
   - `attention_weights`에 dropout layer를 적용한 뒤, `V`와 weighted sum을 계산합니다. 

2. Output projection
   - 모든 head의 attention출력값을 다시 합쳐 `(batch_size, seq_len, embed_dim)`의 shape을 갖도록 만들어줍니다.
     - `torch.Tensor.reshape` ([docs](https://docs.pytorch.org/docs/stable/generated/torch.Tensor.reshape.html)) 함수와 `torch.permute` ([docs](https://pytorch.org/docs/stable/generated/torch.permute.html)) 함수를 이용하세요.
   - 그 후, $\mathbf{W}_O$에 대응하는 선형 변환(`self.out_projection`)을 적용합니다.


In [ ]:
class MultiHeadSelfAttention(nn.Module):
    """
    Multi-Head Self Attention Module.
    """
    def __init__(self, embed_dim, num_heads, dropout_prob = 0.1):
        super().__init__()

        self.embed_dim = embed_dim              # total hidden dimension
        self.num_heads = num_heads              # number of attention heads
        self.head_dim = embed_dim // num_heads  # dimension per head (d_k)
        assert self.head_dim * num_heads == embed_dim, "embed_dim must be divisible by num_heads."

        self.query_projection = nn.Linear(embed_dim, embed_dim)
        self.key_projection = nn.Linear(embed_dim, embed_dim)
        self.value_projection = nn.Linear(embed_dim, embed_dim)
        
        self.out_projection = ... # TODO

        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        """
        Args:
            x: Tensor of shape (batch_size, seq_length, embed_dim)

        Returns:
            attention_output: Tensor of shape (batch_size, seq_length, embed_dim).
        """
        batch_size, seq_length, embed_dim = x.size() 

        ## Project input embeddings into query, key, and value -> obtain (batch_size, seq_len, embed_dim)
        Q = self.query_projection(x)
        K = self.key_projection(x)
        V = self.value_projection(x)

        ## Reshape to split into multiple heads -> obtain (batch_size, seq_len, num_heads, head_dim)
        Q = Q.view(batch_size, seq_length, self.num_heads, self.head_dim)
        K = K.view(batch_size, seq_length, self.num_heads, self.head_dim)
        V = V.view(batch_size, seq_length, self.num_heads, self.head_dim)

        ## Permute to bring head dimension forward -> obtain (batch_size, num_heads, seq_len, head_dim)
        Q = Q.permute(0, 2, 1, 3)
        K = K.permute(0, 2, 1, 3)
        V = V.permute(0, 2, 1, 3)

        ##### YOUR CODE START #####
        ## Scaled dot-product attention for each head


        ## Combine multiple heads and apply output projection



        ##### YOUR CODE END #####

        return attention_output

In [ ]:
embed_dim = 64
num_heads = 8
seq_length = 10
batch_size = 32

mha_module = MultiHeadSelfAttention(embed_dim = embed_dim, num_heads = num_heads, dropout_prob = 0.1)

X = torch.randn(batch_size, seq_length, embed_dim) # (batch_size, seq_length, embed_dim)
attention_output = mha_module(X)

print("MultiHeadSelfAttention output shape:", attention_output.shape)

assert attention_output.shape == (batch_size, seq_length, embed_dim), f"Expected output shape: ({batch_size}, {seq_length}, {embed_dim}), but got {attention_output.shape}"

## Position-wise Feed-Forward Networks

`MultiHeadSelfAttention` 모듈의 출력값에 feed-forward network(FFN)를 적용합니다.
- 이 네트워크는 <b>각 위치(position)별로 독립적</b>으로 작동하며, 시퀀스의 모든 위치에 <b>동일한 변환</b>을 적용합니다. 
- FFN은 Attention이 학습한 관계 표현을 더 복잡하게 변환하여, 모델이 비선형적이고 정교한 패턴을 학습할 수 있도록 돕습니다.

$$ \text{FFN}(\mathbf{x}) = \text{GELU}(\mathbf{x}\mathbf{W}_1+b_1)\mathbf{W}_2+b_2$$

<mark>실습</mark> `FeedForwardNetwork`을 완성하세요.

1. linear layer 1: 임베딩 차원(`hidden_dim`)을 더 큰 차원(`feedforward_dim`)으로 확장합니다.
2. GELU activation: `torch.nn.GELU` ([docs](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html))를 사용하세요
3. dropout layer: 정규화를 위해 `nn.Dropout`을 적용합니다.
4. linear layer 2: 확장된 차원을 다시 원래 임베딩 차원으로 축소합니다.

In [ ]:
class FeedForwardNetwork(nn.Module):
    """
    Position-wise Feed-Forward Networks
    """
    def __init__(self, hidden_dim, feedforward_dim, dropout_prob = 0.1):
        super().__init__()

        self.ffn = nn.Sequential(
            nn.Linear(hidden_dim, feedforward_dim),
            ##### YOUR CODE START #####


            ##### YOUR CODE END #####
        )

    def forward(self, x):
        """
        Args:
            x : Tensor of shape (batch_size, seq_length, hidden_dim)

        Returns:
            out: Tensor of shape (batch_size, seq_length, hidden_dim)
        """
        out = self.ffn(x)
        
        return out

In [ ]:
embed_dim = 64
feedforward_dim = 1024
seq_length = 10       # Sequence length
batch_size = 32       # Batch size

ffn = FeedForwardNetwork(hidden_dim = embed_dim, feedforward_dim = feedforward_dim, dropout_prob = 0.2)

X = torch.randn(batch_size, seq_length, embed_dim) # (batch_size, seq_length, embed_dim)
ffn_output = ffn(X)

print("FeedForwardNetwork output shape:", ffn_output.shape)

assert ffn_output.shape == (batch_size, seq_length, embed_dim), f"Expected output shape: ({batch_size}, {seq_length}, {embed_dim}), but got {ffn_output.shape}"

## TransformerEncoderLayer

앞서 구현한 모듈들을 조합하여 Transformer encoder 레이어를 구현합니다.

<img src="resources/vit_encoder.png" style="width:200px;">

<mark>실습</mark> `TransformerEncoderLayer`을 완성하세요.

1. First Sub-layer (Multi-Head Self Attention):
    - 입력값에 Layer normalization을 적용합니다: `nn.LayerNorm` ([docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html))를 이용하세요
    - 그 결과를 `MultiHeadSelfAttention` 모듈에 통과시키고, dropout을 적용합니다.
    - residual connection: dropout이 적용된 출력값에 이 sub-layer의 입력값을 더해줍니다.

2. Second Sub-layer (Feed-Forward Network):
    - 첫번째 sub-layer의 출력값에 Layer normalization을 적용합니다.
    - `FeedForwardNetwork` 모듈에 통과시키고, dropout을 적용합니다.
    - residual connection: dropout이 적용된 출력값에 이 sub-layer의 입력값을 더해줍니다.


In [ ]:
class TransformerEncoderLayer(nn.Module):
    """
    Single Transformer Encoder Layer.
    """
    def __init__(self, embed_dim, num_heads, feedforward_dim, dropout_prob=0.1):
        super().__init__()

        self.mha = MultiHeadSelfAttention(embed_dim = embed_dim, num_heads = num_heads, dropout_prob = dropout_prob)
        self.ffn = FeedForwardNetwork(hidden_dim = embed_dim, feedforward_dim = feedforward_dim, dropout_prob = dropout_prob)

        self.layer_norm1 = nn.LayerNorm(embed_dim)
        self.layer_norm2 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout_prob)
        self.dropout2 = nn.Dropout(dropout_prob)
        
    def forward(self, x):
        """
        Args:
            x : Tensor of shape (batch_size, seq_length, embed_dim)

        Returns:
            out : Tensor of shape (batch_size, seq_length, embed_dim)
        """

        ##### YOUR CODE START #####
        ## Multi-Head Self Attention
        
        
        ## Feed Forward Network


        ##### YOUR CODE END #####

        return out


In [ ]:
embed_dim = 64
num_heads = 8
feedforward_dim = 1024
seq_length = 10       # Sequence length
batch_size = 32       # Batch size

encoder = TransformerEncoderLayer(embed_dim = embed_dim, num_heads = num_heads, 
                                  feedforward_dim = feedforward_dim, dropout_prob = 0.2)

x = torch.randn(batch_size, seq_length, embed_dim) # (batch_size, seq_length, embed_dim)
encoder_output = encoder(x)
print("TransformerEncoderLayer output shape:", encoder_output.shape)

assert encoder_output.shape == (batch_size, seq_length, embed_dim), f"Expected output shape: ({batch_size}, {seq_length}, {embed_dim}), but got {encoder_output.shape}"

## ImagePatchifier

입력 이미지를 <b>서로 겹치지 않는 패치(patch)</b>들로 분할하고, 이를 Transformer에서 처리할 수 있는 <b>시퀀스(sequence)형태</b>로 변환합니다.

- 입력 이미지 차원: $(C, H, W)$
- 출력 이미지 차원: $(N, C \times P^2)$ 형태로 변환됩니다.
  - $(H, W)$는 원본 이미지의 높이와 너비 입니다. 편의를 위해 $H = W$라고 가정합니다.
  - $C$는 채널 수, $(P, P)$는 패치의 크기입니다.
  - $N = H \times W/P^2$는 생성되는 패치의 개수 입니다.

<mark>실습</mark> `ImagePatchifier`을 완성하세요.

1. <b>Patch Extraction</b>: 이미지를 `(patch_size, patch_size)` 크기의 패치들로 분할합니다.
    - `torch.Tensor.Unfold` ([docs](https://docs.pytorch.org/docs/2.8/generated/torch.Tensor.unfold.html))함수 이용를 이용합니다.
    - 입력 shape : `(batch_size, num_channels, height, width)`
    - 출력 shape : `(batch_size, num_channels, num_patches_h, num_patches_w, patch_size, patch_size)`
2. <b>Flattening</b>: 각 patch들을 1차원 벡터로 펼쳐주고(Flatten), Transformer에 입력할 수 있는 시퀀스 형태로 변환합니다.
    - `view`, `reshape`, `permute`함수를 적절히 활용하세요.
    - 입력 shape : `(batch_size, num_channels, num_patches_h, num_patches_w, patch_size, patch_size)`
    - 출력 shape : `(batch_size, num_patches, num_channels * patch_size * patch_size)`
    - 힌트
      - 여러 패치들을 시퀀스 형태로 Flatten하는 과정은 `(num_patches_h, num_patches_w)` 형태의 텐서를 `(num_patches_h * num_patches_w)`의 형태로 변환하는 것과 동일합니다.
      - 각 패치는 `(num_channels, patch_size, patch_size)`의 shape을 가지는 3차원 텐서이며, 이를 1차원 텐서로 Flatten해줍니다.
      - <mark>주의</mark>: 일반적으로 flatten 순서는 상관없이만, 이번 실습에서는 채점을 위해 height → width 순서를 유지해주세요


In [ ]:
class ImagePatchifier(nn.Module):
    """ Split images into non-overlapping patches and flatten each patch. """
    def __init__(self, image_size, patch_size):
        super().__init__()

        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2
        assert image_size % patch_size == 0, "Image size must be divisible by the patch size."

    def forward(self, x):
        """
        Args:
            x: Tensor of shape (batch_size, num_channels, height, width). Here, height = width = image_size
        Returns:
            flattened_patches: Tensor of shape (batch_size, num_patches, num_channels * patch_size * patch_size)
        """

        batch_size, num_channels, _, _ = x.shape

        ## Unfold image along height and width to extract non-overlapping patches.
        # After two unfolds, (batch_size, num_channels, num_patches_h, num_patches_w, patch_size, patch_size)
        patches = x.unfold(dimension = 2, size = self.patch_size, step = self.patch_size) \
                   .unfold(dimension = 3, size = self.patch_size, step = self.patch_size)
        
        ##### YOUR CODE START #####


        ##### YOUR CODE END #####
        
        return flattened_patches

In [ ]:
batch_size = 64
image_size = 224
patch_size = 16

image_patchfier = ImagePatchifier(image_size = image_size, patch_size = patch_size)
X = torch.randn(batch_size, 3, image_size, image_size) # (batch_size, num_channels, height, width)
flattened_patches = image_patchfier(X)
print("ImagePatchifier output shape:", flattened_patches.shape)

assert flattened_patches.shape == (batch_size, (image_size / patch_size) ** 2, 3 * patch_size * patch_size), f"Expected output shape: ({batch_size}, {(image_size / patch_size) ** 2}, {3 * patch_size * patch_size}), but got {flattened_patches.shape}"

`ImagePatchifier` 모듈을 잘 구현하셨다면, 아래 코드를 통해 <b>새의 이미지</b>가 패치 단위로 분할된 결과를 시각적으로 확인할 수 있습니다.

In [ ]:
def visualize_patches(image, patch_size):
    patchifier = ImagePatchifier(image_size = 224, patch_size = patch_size)

    patches = patchifier(image)[0]
    num_patches = patches.shape[0]
    grid_size = int(np.sqrt(num_patches))

    fig, axes = plt.subplots(grid_size, grid_size, figsize=(8, 8))
    for idx, patch in enumerate(patches):
        row = idx // grid_size
        col = idx % grid_size
        ax = axes[row, col]
        
        patch = patch.view(-1, patch_size, patch_size).permute(1, 2, 0).numpy()
        ax.imshow(patch)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


img = Image.open('resources/n01580077_1031.JPEG')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
img_tensor = transform(img).unsqueeze(0)  # (1, num_channels, height, width)

visualize_patches(image = img_tensor, patch_size = 16)

## PatchEmbedding
`ImagePatchifier`에서 얻은 시퀀스 형태의 패치들을 임베딩 벡터(embedding vector)로 변환한 후, `[CLS]`토큰과, positional embedding을 추가합니다.

<center><img src="resources/vit_model.png" style="width:500px;"></center>

1. `ImagePatchifier`의 출력:

   $$[\mathbf{x}_1, \mathbf{x}_2, ..., \mathbf{x}_N] \in \mathbb{R}^{N\times (P^2 \cdot C)}$$

   - $C$는 채널 수, $(P, P)$는 패치의 크기입니다.
   - $N = H \cdot W/P^2$는 패치의 개수 입니다. ($(H, W)$는 원본 이미지의 높이와 너비)

2. Patch embedding

   - 각 이미지 패치를 $d_{embed}$차원으로 선형변환합니다.
        $$[\mathbf{x}_1 \mathbf{E},\mathbf{x}_2 \mathbf{E}, ..., \mathbf{x}_N \mathbf{E}] \in \mathbb{R}^{N\times d_{embed}}$$

        - $\mathbf{E} \in \mathbb{R}^{(P^2 \cdot C) \times d_{embed}}$ 는 학습가능한 선형 변환 행렬

3. Prepend `[CLS]` token

   - `[CLS]`는 이미지 분류를 위한 특별 토큰으로, 이 토큰에 대응하는 학습 가능한 임베딩 벡터 $\mathbf{x}_{CLS}$를 시퀀스의 맨 앞에 추가합니다.
        $$[\mathbf{x}_{CLS}, \mathbf{x}_1 \mathbf{E},\mathbf{x}_2 \mathbf{E}, ..., \mathbf{x}_N \mathbf{E}] \in \mathbb{R}^{(N+1) \times d_{embed}}$$

        - `[CLS]` 토큰에 대응하는 Tranformer encoder의 최종 출력은 이미지(시퀀스) 전체의 표현을 담도록 학습되며, 여기에 MLP head를 연결하여 이미지 분류 작업을 수행합니다

4. Position embeddings

   - 학습가능한(learnable) Positional Embedding $\mathbf{E}_{pos}$을 더해줍니다. 이를 통해 모델은 각 패치의 위치를 구분하고, 패치들 간의 공간적 관계를 이해할 수 있습니다.
        $$[\mathbf{x}_{CLS}, \mathbf{E},\mathbf{x}_2 \mathbf{E}, ..., \mathbf{x}_N \mathbf{E}] + \mathbf{E}_{pos}$$
        - $\mathbf{E}_{pos} \in \mathbb{R}^{(N+1)\times d_{embed}}$



<mark>실습</mark> 위 설명을 참고하여 `PatchEmbedding`을 완성하세요.

In [ ]:
class PatchEmbedding(nn.Module):
    """
    Project flattened image patches into an embedding space, prepend a classification token,
    and adds learnable positional embeddings.
    """
    def __init__(self, num_patches, patch_size, embed_dim, num_channels = 3):
        super().__init__()
        self.num_patches = num_patches
        self.patch_size = patch_size
        self.embed_dim = embed_dim
        self.num_channels = num_channels

        self.patch_projection = ... # TODO
        
        self.cls_token = nn.Parameter(torch.empty(1, 1, embed_dim)) # learnable classification token
        self.position_embeddings = nn.Parameter(torch.empty(1, num_patches + 1, embed_dim)) # learnable positional embeddings

        self._initialize_parameters()

    def _initialize_parameters(self):
        nn.init.trunc_normal_(self.cls_token, std=0.02)
        nn.init.trunc_normal_(self.position_embeddings, std=0.02)

    def forward(self, flattened_patches):
        """
        Args:
            flattened_patches: Tensor of shape (batch_size, num_patches, num_channels * patch_size * patch_size)
        Returns:
            embeddings: Tensor of shape (batch_size, num_patches + 1, embed_dim)
        """
        batch_size, num_patches, _ = flattened_patches.shape

        ## Project patches to embedding dimension
        patch_embeddings = self.patch_projection(flattened_patches)  # (batch_size, num_patches, embed_dim)

        ## Broadcast [CLS] token to batch size
        cls_tokens = self.cls_token.expand(batch_size, -1, -1)  # (batch_size, 1, embed_dim)

        ##### YOUR CODE START #####
        ## Concatenate class token with patch embeddings


        ## Add positional embeddings


        ##### YOUR CODE END #####

        return embeddings

In [ ]:
batch_size = 64
image_size = 224
patch_size = 16
num_patches = (image_size // patch_size) ** 2
embed_dim = 512

to_patch_embedding = PatchEmbedding(num_patches = num_patches, patch_size = patch_size, embed_dim = embed_dim)
X = torch.randn(batch_size, num_patches, 3 * patch_size * patch_size) # (batch_size, num_patches, num_channels * patch_size * patch_size)

patch_embeddings = to_patch_embedding(X)
print("PatchEmbedding output shape:", patch_embeddings.shape)

assert patch_embeddings.shape == (batch_size, num_patches + 1, embed_dim), f"Expected output shape: ({batch_size}, {num_patches + 1}, {embed_dim}), but got {patch_embeddings.shape}"

## ViT (Vision Transformer)
지금까지 구현한 모듈들을 모두 조합하여 ViT 아키텍쳐를 완성합니다.

<mark>실습</mark> `ViT`를 완성하세요.
1. <b>Patchification</b>: `ImagePatchifier`를 사용해 이미지를 겹치지 않는 패치들로 분할하고, 시퀀스 형태로 변환합니다.
2. <b>Patch Embedding</b>: `PatchEmbedding` 모듈을 각 패치를 임베딩 벡터로 변환합니다.
3. <b>Transformer Encoder</b>: `TransformerEncoderLayer`를 `num_transformer_layers`만큼 통과시킵니다.
4. <b>Classification Head</b>
    - `[CLS]` token에 대응되는 Transformer Encoder의 출력값을 추출하여, 이미지 전체를 대표하는 임베딩으로 사용합니다.
    - 해당 임베딩에 mlp_head (`nn.Linear`)를 적용하여 최종 분류 결과(logits) 값을 얻습니다.

In [ ]:
class ViT(nn.Module):
    """
    Vision Transformer (ViT) model.
    """
    def __init__(self, image_size, patch_size, num_channels, embed_dim,
                 num_transformer_layers, num_heads, feedforward_dim, 
                 num_classes, dropout_prob=0.1):
        
        super().__init__()

        self.patchifier = ImagePatchifier(image_size = image_size, patch_size = patch_size)
        self.to_patch_embedding = PatchEmbedding(self.patchifier.num_patches, patch_size, embed_dim, num_channels)

        self.transformer_encoder = nn.ModuleList([
            TransformerEncoderLayer(embed_dim, num_heads, feedforward_dim, dropout_prob)
            for _ in range(num_transformer_layers)
        ])

        self.layer_norm = nn.LayerNorm(embed_dim)
        self.mlp_head = ... # TODO

        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, images):
        """
        Args:
            images: Tensor of shape (batch_size, num_channels, height, width)
        Returns:
            logits: Tensor of shape (batch_size, num_classes)
        """
        patches = self.patchifier(images) # (batch_size, num_patches, num_channels * patch_size * patch_size)
        embeddings = self.to_patch_embedding(patches)  # (batch_size, 1 + num_patches, hidden_dim)
        embeddings = self.dropout(embeddings)
        
        for layer in self.transformer_encoder:
            embeddings = layer(embeddings)
        embeddings = self.layer_norm(embeddings) # (batch_size, num_patches + 1, embed_dim)

        ##### YOUR CODE START #####


        ##### YOUR CODE END #####
  
        return logits 

In [ ]:
image_size = 224
patch_size = 16
num_channels = 3
embed_dim = 1024
num_transformer_layers = 12
num_heads = 8
feedforward_dim = 3072
num_classes = 1000
batch_size = 4


model = ViT(image_size, patch_size, num_channels, embed_dim,
            num_transformer_layers, num_heads, feedforward_dim, num_classes)

X = torch.randn(batch_size, num_channels, image_size, image_size) # (batch_size, num_channels, height, width)
logits = model(X)
print("ViT output shape:", logits.shape)

assert model(X).shape == (batch_size, num_classes), f"Expected output shape: ({batch_size}, {num_classes}), but got {logits.shape}"
assert sum(p.numel() for p in model.parameters()) == 127993832, f"Expected number of model parameter: 86567656, but got {sum(p.numel() for p in model.parameters())}"

print("\033[92m All test passed!")

## Training 

<mark>실습</mark> 완성한 ViT 모델을 CIFAR-10 데이터셋을 이용하여 학습해봅니다.

In [ ]:
def train_main():
    ## data and preprocessing settings
    data_root_dir = '/datasets'
    num_workers = 4

    ## Training Hyperparameters
    num_epochs = 5
    batch_size = 64  # Comsumes 1.6GB of GPU memory
    learning_rate = 1e-3

    ## Model hyperparameters
    image_size = 32
    patch_size = 4
    num_channels = 3
    embed_dim = 256
    num_transformer_layers = 8
    num_heads = 4
    feedforward_dim = 1024
    dropout_prob = 0.1


    device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
    print(f"Using {device} device")

    ## Data loaders
    train_dataloader, val_dataloader, test_dataloader, num_classes = load_cifar10_dataloaders(
        data_root_dir, device, batch_size = batch_size, num_worker = num_workers)
    
    ## Model, Loss, Optimizer
    model = ViT(image_size, patch_size, num_channels, embed_dim,
                num_transformer_layers, num_heads, feedforward_dim, 
                num_classes, dropout_prob = dropout_prob).to(device)
    
    print(f"Number of model parameters: {sum(p.numel() for p in model.parameters())}" +
          f" ({sum(p.numel() for p in model.parameters() if p.requires_grad)} trainable)")

    criterion = nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    ## Training loop
    for epoch in range(num_epochs):
        train_loss = train_one_epoch(model, device, train_dataloader, criterion, optimizer, epoch)
        val_loss, val_accuracy = evaluate_one_epoch(model, device, val_dataloader, criterion, epoch)
        print(f"[Epoch {epoch+1:>2}/{num_epochs:<2}] Train Loss: {train_loss:>8.4f}" +
              f" | Val Loss: {val_loss:>8.4f} | Val Accuracy: {val_accuracy * 100:>4.1f} %")
        
    ## Test the model
    test_loss, test_accuracy = evaluate_one_epoch(model, device, test_dataloader, criterion, epoch)

    print(f"Test Loss: {test_loss:>8.4f} | Test Accuracy: {test_accuracy * 100:>4.1f} %")

In [ ]:
train_main()

코드 구현이 잘 되었다면 별도의 하이퍼파라미터 튜닝(hyperparameter tuning)없이 `Validation Accuracy > 35%`를 달성하실 수 있습니다